In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv("./DASS_data_21.02.19/data.csv", sep=r'\t', engine='python')

dataset

In [ ]:
print(dataset.columns,"\n\n")
print(dataset.shape,"\n\n")
print(dataset.dtypes,"\n\n")

In [ ]:
dataset.isnull()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [ ]:
k=0
for i in dataset.columns:    
    print(str(k)+" :"+i)
    k+=1

In [ ]:
#Preprocessing
    

In [ ]:
x_questions=dataset.iloc[:,:126]
x_personal=dataset.iloc[:,157:171]
y=dataset.iloc[:,134]


print(x_questions.isnull().sum().sum())
print(x_personal.isnull().sum().sum())
print(y.isnull().sum().sum())

In [ ]:

l=[]
for i in range(1,43):
    l.append((3*i)-2)
   
    
x_questions.drop(x_questions.columns[l],axis=1,inplace=True)
x_questions

In [ ]:
x_answer=x_questions.iloc[:,0::2]
x_time=x_questions.iloc[:,1::2]
x_time=x_time/1000
min_t=x_time.min()
max_t=x_time.max()

range_t=max_t-min_t

x_time.describe()

In [ ]:

sub_value=((x_time-min_t)/range_t)
sub_value.describe()

In [ ]:
x_answer=x_answer-sub_value.values

In [ ]:
x_answer

In [ ]:
###Preprocessing x_personal dataframe


x_personal

In [ ]:
x_personal.drop(['engnat','screensize','uniquenetworklocation','hand'],axis=1,inplace=True)

In [ ]:
x_personal

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


enc = OneHotEncoder()

# 2. FIT
enc.fit(x_personal.iloc[:,[1,2,4,5,6]])

# 3. Transform
onehotlabel = enc.transform(x_personal.iloc[:,[1,2,4,5,6]])

onehotlabels.shape

In [ ]:
x_personal.drop(x_personal.columns[[1,2,4,5,6]],axis=1,inplace=True)




x_personal.describe()


In [ ]:
x_personal=x_personal[x_personal['age']<100]
x_personal=x_personal[x_personal['familysize']<10]
x_personal.shape

In [ ]:




onehotlabels_df=pd.DataFrame(onehotlabels)

#pd.concat([x_personal,onehotlabels_df])
x_personal=pd.merge(x_personal,onehotlabels_df,left_index=True,right_index=True)
x_personal

In [ ]:
#MERGING X_personal and x_answers

x=pd.merge(x_answer,x_personal,left_index=True,right_index=True)
x

In [ ]:
x.describe()

In [ ]:
preprocessed_data=pd.merge(x,y,left_index=True,right_index=True)

preprocessed_data

In [ ]:
y.value_counts().plot(kind='pie', autopct='%1.0f%%')

In [ ]:
x=preprocessed_data.iloc[:,:-1]
y=preprocessed_data.iloc[:,-1]

x,y

In [ ]:
y = y.replace([5, 4, 3, 2, 1], 0)
y = y.replace([7, 6], 1)

In [ ]:
y.describe()

In [ ]:
#SPLIT THE DATASET

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [ ]:
#FEATURE SCALING

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

mm_scale=preprocessing.MinMaxScaler(feature_range=(0,1))

x_train=mm_scale.fit_transform(x_train)
x_test=mm_scale.transform(x_test)


len(x_train[0]),len(x_train)

In [ ]:
#PCA


from sklearn.decomposition import PCA

pca1=PCA(0.95)
x_pca=pca1.fit_transform(x_train)
x_pca_test=pca1.transform(x_test)

pca1.n_components_

In [ ]:
print(pca1.explained_variance_ratio_.sum())
print(pca1.components_.shape)
print(pca1.explained_variance_ratio_)


In [ ]:
x_pca

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression(random_state=0)
classifier1.fit(x_pca,y_train)

y_pred1 = classifier1.predict(x_pca_test)


from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred1))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred1)
cm


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=20,tol=np.infty)
sgd_clf.fit(x_pca,y_train)


y_pred = sgd_clf.predict(x_pca_test)


from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm


In [ ]:
from sklearn import tree

classifier2  = tree.DecisionTreeClassifier()
classifier2.fit(x_pca,y_train)

predictions = classifier2.predict(x_pca_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

from sklearn.metrics import confusion_matrix


cm=confusion_matrix(y_test,predictions)
cm

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=4, random_state=0)

clf.fit(x_pca, y_train)

predictions = clf.predict(x_pca_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions))

from sklearn.metrics import confusion_matrix


cm=confusion_matrix(y_test,predictions)
cm